In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import glob as gb

In [ ]:
PERF_DIR = Path('/Users/julien/Software/Others/OS-build-release/results/')

In [ ]:
csvs = gb.glob(str(PERF_DIR / '*.csv'))

In [ ]:
csv = csvs[0]

In [ ]:
def find_skip_rows(results_file: Path) -> int:
    """
    Open the result benchmark, and locate the actual start of the timings

    Returns:
    --------
    skiprows (int): the number of lines to skip when read_csv is called
    """
    skiprows = 0
    search_str = 'name,iterations,'
    with open(results_file, 'r') as f:
        content = f.read()
    if search_str not in content:
        return None
    lines = content.splitlines()
    while (search_str not in lines[skiprows]):
        skiprows += 1

    return skiprows

In [ ]:
def read_bench_results(results_file: Path) -> pd.DataFrame:
    skiprows = find_skip_rows(csv)
    if skiprows is None:
        return None
    df = pd.read_csv(csv, skiprows=skiprows, index_col=0)
    df = df.loc[df['iterations'].notnull()]
    # Convert everything in ms
    for col in ['real_time', 'cpu_time']:
        df[col] = (df[[col, 'time_unit']].apply(
            lambda row: pd.to_timedelta(arg=row[0], unit=row[1]), axis=1)
                   .dt.total_seconds() * 1e3)
        
    return df

In [ ]:
df_dict = {}
dfs = []
for csv in csvs:
    fname = Path(csv).name
    #if 'Model_ModelObjects' in fname:
    #    fname = 'Model_ModelObjects'
    prefix  = Path(csv).name.split('_')[0]
    test = '_'.join(fname.split('_')[1:-1])
    
    df = read_bench_results(csv)
    if df is None:
        continue
    df['test_file'] = test
    df['prefix'] = prefix
    dfs.append(df)

In [ ]:
df = pd.concat(dfs, axis=0)

In [ ]:
df.set_index(['prefix', 'test_file'], append=True, inplace=True)

In [ ]:
df_real = df['real_time'].unstack('prefix')

In [ ]:
df_real.loc[[x for x in df_real.index if 'VT' in x[0]]]

In [ ]:
df_real[df_real['move1'].isnull()].index.tolist()

In [ ]:
df_sum = df_real[df_real['move1'].notnull()].groupby('test_file').sum()
df_sum['orim'] = df_sum[['ori', 'ori2']].mean(axis=1)

In [ ]:
df_sum[['orim', 'move1']].groupby('test_file').sum().pct_change(axis=1).iloc[:, [-1]].style.format('{:.2%}')

In [ ]:
df_sum['move1'].subtract(df_sum[['ori', 'ori2']].mean(axis=1))

In [ ]:
df_real[['ori', 'move1']].groupby('test_file').sum().pct_change(axis=1).iloc[:, [-1]].style.format('{:.2%}')

In [ ]:
df_real.plot(figsize=(16, 9))

In [ ]:
df_real['orim'] = df_real[['ori', 'ori2']].mean(axis=1)
df_real['newm'] = df_real[['new'
                           #, 'new2'
                          ]].mean(axis=1)

In [ ]:
(df_real[['orim', 'newm']]).pct_change(axis=1)['newm'].sort_values().plot(kind='barh', figsize=(16, 36))

In [ ]:
(df_real['newm'] - df_real['orim']).sort_values().plot(kind='barh', figsize=(16, 9))

In [ ]:
df_real[['orim', 'newm']].plot(kind='barh', figsize=(16, 36))

In [ ]:
df_real[['orim', 'newm']].mean()